In [0]:
class TopicBatchProcessor:    
    def __init__(self, s3_connector_instance, topic_name):
        self._topic_name = topic_name
        self._file_location = f"{s3_connector_instance._bucket_mount_name}/topics/{self._topic_name}/partition=0/*.json"
        self._extracted_batch_data = None # return to this - might want to timestamp batches depending on workflow
        self._cleaned_batch_data = None 
    
    def read_json_files_from_s3_bucket_to_df(self) -> None:
        file_type = "json"
        infer_schema = "true"
        df = spark.read.format(file_type) \
                        .option("inferSchema", infer_schema) \
                        .load(self._file_location)

        self._extracted_batch_data = df
    
    def clean_dataframe(self, cleaning_function) -> None:
        if self._extracted_batch_data is not None:
            try:
                no_duplicate_records_df = self._extracted_batch_data.distinct() # maybe not needed?
                cleaned_df = cleaning_function(no_duplicate_records_df)
                self._cleaned_batch_data = cleaned_df.cache()
            except Exception as e:
                print(f"Error occured during cleaning of {self._topic_name} extracted data: {e}")
        else:
            print(f"Error occured during cleaning of {self._topic_name} extracted data:\
                    no dataframe of extracted data saved to object instance.")     